================================================================================
Multi-cell Massive MIMO Downlink Scenario in Python.
--

================================================================================

Based on:
--

Emil Bjornson, Jakob Hoydis and Luca Sanguinetti (2017),

"Massive MIMO Networks: Spectral, Energy, and Hardware Efficiency",
Foundations and Trends in Signal Processing.

Installation
--

In [1]:
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install statsmodels
!pip install tensorflow==2.3.0

# for convex function
!pip install cvxpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 821 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320.4 MB 42 kB/s 
     |████████████████████████████████| 26.1 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 50.6 MB/s 
     |████████████████████████████████| 459 kB 66.9 MB/s 
     |████████████████████████████████| 20.1 MB 35.9 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import
--

In [2]:
import sys
import gym
import math
import time
import pylab
import cmath
import random
import itertools
import threading
import tensorflow
import scipy as sp
import numpy as np
import pandas as pd
from gym import Env
import scipy.io as sc
import tensorflow as tf
from sys import version
from absl import logging
from numpy import ndarray
from scipy import special
import statsmodels.api as sm
from gym.utils import seeding
from scipy.constants import *
from rl.agents import DQNAgent
from scipy.special import erfinv
from scipy.integrate import quad
from scipy.linalg import toeplitz
from numba import jit, njit, prange
from gym.spaces import Discrete, Box
from collections import deque, Counter
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from mpl_toolkits.mplot3d import Axes3D
from gym import Env, error, spaces, utils
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input

# Set of times new roman font.
from matplotlib import cm
from matplotlib import colors
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

dtype = np.float32

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

##############################################################################
import cvxpy as cp
from jax import jacrev
import jax.numpy as jnp

from scipy.linalg import sqrtm
import copy

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:513: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/tensor_util.py:107: DeprecationWarning: `np.object` is a deprecated ali

NEW Multi-cell Massive MIMO scenario at 10.21 (FRI)
--


In [5]:
class M_MIMOEnv(Env):
  def __init__(self, N, M, K):

    self.N = N          # Number of cells, it is equals to number of BSs
    self.M = M          # Number of BS transmission antennas
    self.K = K          # Number of UEs in a cell
    self.BW = 20e6      # Bandwidth = 20MHz
    self.NF = 7         # Power of noise figure [dBm]
    self.Ns = 10        # Number of sample
    self.min_p = -20    # Minimum transmission power [dBm]
    self.max_p = 23     # Maximum transmission power [dBm]
    self.num_p = 10     # Number of action space

    # Defined the pilot reuse factor
    self.pilot_reuse_factor = 1

    # Select the number of channel realizations per setup
    self.nbrOfRealizations = 100

    # Total uplink transmission power per UE (mW)
    self.p = 100

    # Total downlink transmission power per UE (mW)
    self.rho = 100

    # Maximum downlink tranmsit power per BS (mW)
    self.Pmax = self.K * self.rho

    # Compute downlink power per UE in case of equal power allocation
    self.rhoEqual = (self.Pmax / self.K) * np.ones((self.K, self.N))

    # Compute noise power
    self.noiseVariancedBm = -174 + 10 * np.log10(self.BW) + self.NF

    # Select length of coherence block
    self.tau_c = 200

  # Generate random seed
  def randn2(self, *args, **kargs):
    args_r = tuple(reversed(args))
    uniform = np.random.rand(*args_r)
    untiform = uniform.transpose()
    
    return np.sqrt(2) * erfinv(2 * uniform - 1)


  # Generate hermitian matrix
  def hermitian(self, X):
    return X.conj().swapaxes(-1, -2)


  # Divide function by using hermitian.
  def mldivide(self, A: ndarray, B: ndarray, A_is_hermitian=False):
    if A_is_hermitian:
      return self.hermitian(np.linalg.solve(A, self.hermitian(B)))
    else:
      return self.hermitian(np.linalg.solve(self.hermitian(A), self.hermitian(B)))


  # Correlation in term of real signal parts.
  @njit
  def correlation_real(self, x, antenna_spacing, col): 
    return np.cos(2 * np.pi * antenna_spacing * col * np.sin(x))


  # Correlation in term of imaginary signal parts.
  @njit
  def correlation_imag(self, x, antenna_spacing, col): 
    return np.sin(2 * np.pi * antenna_spacing * col * np.sin(x))


  # Probability density function (PDF) : gussian
  @njit
  def gaussian_pdf(self, x, mean, dev): 
    return np.exp(-(x-mean) ** 2 / (2 * dev ** 2)) / (np.sqrt(2 * np.pi) * dev)


  # Correlation function
  @njit
  def corr(self, x, theta, asd, antenna_spacing, dist, col, real_imag):
    if real_imag == 0:
        res = np.cos(2 * np.pi * antenna_spacing * col * np.sin(x))
    else:
        res = np.sin(2 * np.pi * antenna_spacing * col * np.sin(x))
    if dist =='gaussian':
        res *= self.gaussian_pdf(x, theta, asd)
    return res


  # Convert dBm to watts
  def dBm2Watts(self, P):
    # P_watts = (10 ** ((P - 30) / 10))
    P_watts = (10.0 ** (P/10.0)) * 1e-3
    return P_watts


  # local scattering channel model. where the local is a single cell in total cell
  def R_local_scattering(self, M, theta, asd_deg, antenna_spacing=0.5, dist='Gaussian', accuracy=2, dtype=complex):
    # In radians
    asd = asd_deg * np.pi / 180

    # correlation matrix is Toeplitz structure, so only need first row
    first_row = np.zeros([M,], dtype=dtype)

    if accuracy == 1:
        lb = None
        ub = None

        dist = dist.lower()
        if dist == 'gaussian':
            lb = theta - 20 * asd
            ub = theta + 20 * asd

        else:
            raise NotImplementedError

        for col in range(0, M):
            # distance from the first antenna
            c_real:float = quad(func=self.corr, a=lb, b=ub, args=(theta, asd, antenna_spacing, dist, col, 0))[0]
            c_imag:float = quad( func=self.corr, a=lb, b=ub, args=(theta, asd, antenna_spacing, dist, col, 1))[0]

            first_row[col] = complex(c_real, c_imag)
    elif accuracy == 2:
        # Gaussian distribution
        distance = np.arange(M)
        x1 = np.exp(1j * 2 * np.pi * antenna_spacing * np.sin(theta) * distance)
        x2 = np.exp(-asd ** 2 / 2 * (2 * np.pi * antenna_spacing * np.cos(theta) * distance) ** 2)
        first_row = x1 * x2

    return toeplitz(c=first_row.conjugate())


  # Channel statistics between UE's at random locations and the BS.
  def channel_stat_setup(self, N, K, M, asd_degs, no_BS_per_dim=None, accuracy=2,):
    # Square side, in meters
    side_length = 1000

    # Pathloss exp
    alpha = 3.76

    # Average channel gain in dB at the ref. distance 1 meter. At exponent set to 3.76, at 1km it's -148.1 dB
    constant_term = -35.3

    # Standard deviation of shadow fading
    sigma_sf = 0

    # Minimum distance between BS and UEs
    min_UE_BS_dist = 35
    
    # Antenna spacing of wavelengths
    antenna_spacing = 0.5
    if no_BS_per_dim is None:
        no_BS_per_dim = np.array([np.sqrt(N), np.sqrt(N)])
    inter_bs_distance = side_length / no_BS_per_dim

    # Scatter the BSs
    BS_positions = np.stack(np.meshgrid(np.arange(inter_bs_distance[0]/2, side_length, inter_bs_distance[0]),np.arange(inter_bs_distance[1]/2, side_length, inter_bs_distance[1]),indexing='ij'),axis=2).reshape([-1,2])
    
    # Now all the other nine alternatives of the BS locations
    wrap_locations = np.stack(np.meshgrid(np.array([-side_length, 0, side_length]), np.array([-side_length, 0, side_length]), indexing='ij' ), axis=2).reshape([-1,2])

    # For each BS locations, there are 9 possible alternative locations including the original one. Here uses broadcasting to add (9,2) to a (num_BS, 1, 2) to get a (num_BS, 9, 2)
    BS_positions_wrapped = np.expand_dims(BS_positions, axis=1) + wrap_locations

    UEpositions = np.zeros([K, N, 2])
    perBS = np.zeros([N,], dtype=int)

    # Normalized spatial correlation matrices
    R = np.zeros([M, M, K, N, N, len(asd_degs)], dtype=complex)
    
    # Prepare to store average channel gain numbers (in dB)
    self.channel_gain = np.zeros([K, N, N])

    # Go through all the cells
    for i in range(N):
        # Put K UEs in the cell, uniformly. UE's not satisfying the minimum distance are replaced
        res = []
        while perBS[i] < K:
            UEremaining = K - perBS[i]
            pos = np.random.uniform(-inter_bs_distance/2, inter_bs_distance/2,size=(UEremaining, 2))
            cond = np.linalg.norm(pos, ord=2, axis=1) >= min_UE_BS_dist

            # Satisfying minimum distance with respect to BS shape
            pos = pos[cond, :]
            for x in pos:
                res.append(x + BS_positions[i])
            perBS[i] += pos.shape[0]
        UEpositions[:, i, :] = np.array(res)

        # Loop through all BS for cross-channels
        for j in range(N):
            # distance between all UEs in cell i to BS j, considering wrap-around.
            dist_ue_i_j = np.linalg.norm(np.expand_dims(UEpositions[:, i], axis=1) - BS_positions_wrapped[j, :, :], axis=2)
            dist_bs_j = np.min(dist_ue_i_j, axis=1)
            which_pos = np.argmin(dist_ue_i_j, axis=1)

            # Average channel gain with large-scale fading mdoel in (2.3), neglecting shadow fading
            self.channel_gain[:, i, j] = constant_term - alpha * 10 * np.log10(dist_bs_j)

            # Generate spatial correlation matrices for channels with local scattering model
            for k in range(K):
                vec_ue_bs = UEpositions[k, i] - BS_positions_wrapped[j, which_pos[k]]
                angle_BS_j = np.arctan2(vec_ue_bs[1], vec_ue_bs[0])

                for spr, asd_deg in enumerate(asd_degs):
                  R[:, :, k, i, j, spr] = self.R_local_scattering(M, angle_BS_j, asd_deg, antenna_spacing, accuracy=accuracy)

        # Go through all UEs in cell l and generate shadow fading realizations
        for k in range(K):
            # See if another BS has a larger avg. channel gain to the UE than BS i
            while True:
                # Generate new shadow fading realizations until all UE's in cell i has its largest avg. channel gain from BS i
                shadowing = sigma_sf * self.randn2(N)
                # shadowing = sigma_sf * np.random.randn(N)
                channel_gain_shadowing = self.channel_gain[k, i] + shadowing
                if channel_gain_shadowing[i] >= np.max(channel_gain_shadowing):
                    break
            self.channel_gain[k,i,:] = channel_gain_shadowing

    return R, self.channel_gain

  def MMSE_channel_estimates(self, R, channel_gain_db):
    # Generate uncorrelated Rayleigh fading channel realizations
    H = (np.random.randn(self.M,self.nbrOfRealizations,self.K,self.N,self.N) + 1j * np.random.randn(self.M,self.nbrOfRealizations,self.K,self.N,self.N))

    # Prepare a matrix to save the channel gains per UE
    betas = np.zeros((self.K,self.N,self.N))
    R_gain = np.zeros(R.shape, dtype=complex)

    # Go through all channels and apply the channel gains to the spatial correlation matrices
    for j in range(self.N):
      for l in range(self.N):
        for k in range(self.K):
          if channel_gain_db[k, j, l] > -np.inf:

            # Extract channel gain in linear scale
            betas[k,j,l] = 10 ** (channel_gain_db[k, j, l] / 10)
            R_gain[:, :, k, j, l] = betas[k, j, l] * R[:, :, k, j, l]

            # Apply correlation to the uncorrelated channel realizations
            Rsqrt = sp.linalg.sqrtm(R_gain[:, :, k, j, l])
            H[:, :, k, j, l] = np.sqrt(0.5) * Rsqrt @ H[:, :, k, j, l]

          else:
            betas[k,j,l] = 0
            R_gain[:, :, k, j, l] = 0
            H[:, :, k, j, l] = 0

    # do the channel estimation
    self.tau_p = self.pilot_reuse_factor * self.K

    # pilot reuse patterns. only work when there are 16 BS
    if self.pilot_reuse_factor == 1:
      pilot_pattern = np.zeros((self.N,))
      # pilot_pattern = np.zeros((self.N, 1))

    elif self.pilot_reuse_factor == 2:
      pilot_pattern = np.kron(np.ones((2,)), np.array([0,1,0,1,1,0,1,0]))
        
    elif self.pilot_reuse_factor == 4:
      pilot_pattern = np.kron(np.ones((2,)), np.array([0,1,0,1,2,3,2,3]))

    elif self.pilot_reuse_factor == 16:
      pilot_pattern = np.arange(self.N)

    else:
      raise NotImplementedError('Unknown f')

    # realizations of normalized noise
    Np = np.sqrt(0.5) * (np.random.randn(self.M,self.nbrOfRealizations,self.K,self.N, self.pilot_reuse_factor) + 1j * np.random.randn(self.M,self.nbrOfRealizations,self.K,self.N, self.pilot_reuse_factor))

    # Prepare to MMSE, LS channel estimation
    Hhat_MMSE = np.zeros((self.M, self.nbrOfRealizations, self.K, self.N, self.N), dtype=complex)

    # Go through all cells
    for j in range(self.N):
      
      for g in range(self.pilot_reuse_factor):
        # Extract the cells that belong to pilot group g
        group_members = np.nonzero(g == pilot_pattern)[0]

        # Compute processed pilot signal for all UEs that use these pilots, according to (3.5)
        yp = np.sqrt(self.p) * self.tau_p * np.sum(H[:, :, :, group_members, j], axis=3) + np.sqrt(self.tau_p) * Np[:, :, :, j, g]

        for k in range(self.K):
          # Compute the matrix that is inverted in the MMSE estimator, where the np.eye(self.M) for store identity matrix of size M x M
          PsiInv = self.p * self.tau_p * np.sum(R_gain[:, :, k, group_members, j], axis=2) + np.eye(self.M)

          for l in group_members:
            # MMSE (minimum mean-square estimation) estimation method
            RPsi = np.linalg.solve(PsiInv.conjugate().transpose(), R_gain[:, :, k, l, j]).conjugate().transpose()
            Hhat_MMSE[:, :, k, l, j] = np.sqrt(self.p) * RPsi @ yp[:, :, k]

    return H, Hhat_MMSE

  def compute_SINR_DL_rzf_precoding(self, H, Hhat, nbrOfRealizations):
    # Store identity matrices of different sizes
    eyeK = np.eye(self.K)

    # Prepare to store simulation results for signal gains
    signal_RZF = np.zeros((self.K, self.N), dtype=complex)

    # Prepare to store simluation results for Bernoulli_extra_interference powers
    interf_RZF = np.zeros((self.K, self.N, self.K, self.N))

    # Go through all channel realizations
    for n in range(nbrOfRealizations):
      # Go through all cells
      for j in range(self.N):
        # Extract channel realizations from all UEs to BS j
        Hallj = np.reshape(H[:,n,:,:,j], (self.M, self.K * self.N), 'F')

        # Extract channel realizations from all UEs to BS j
        Hhatallj = np.reshape(Hhat[:,n,:,:,j], (self.M, self.K * self.N), 'F')

        # Compute MR combining in (4.11)
        V_MR = Hhatallj[:, self.K * j:self.K * (j + 1)]

        # Compute RZF combining in (4.9)
        temp = self.p*np.matmul(np.matrix(V_MR).getH(), V_MR) + eyeK
        temp = np.linalg.inv(temp)
        V_RZF = np.matmul(self.p * V_MR, temp)

        for k in range(self.K):
          if np.linalg.norm(V_MR[:,k])>0:
            # RZF precoding
            w = V_RZF[:,k]/np.linalg.norm(V_RZF[:,k]) # Extract precoding vector
            w = np.reshape(w, (1,self.M)).conj() # Hermitian: make it a row vector and conjugate

            # Compute realizaqtions of the terms inside the expectations of the signal and Bernoulli_extra_interference temrs of (7.2) and (7.3)
            h_temp = H[:,n,k,j,j]
            signal_RZF[k,j] = signal_RZF[k,j] + (np.inner(w,h_temp))/nbrOfRealizations
            h_temp = np.matmul(w,Hallj)
            h_temp = np.abs(np.array(h_temp))**2
            h_temp = np.reshape(h_temp, (self.K, self.N), 'F')

            interf_RZF[k,j,:,:] = interf_RZF[k,j,:,:] + h_temp/nbrOfRealizations

    # Compute the terms in (7.2)
    signal_RZF = np.abs(signal_RZF)**2

    # Compute the terms in (7.3)
    for j in range(self.N):
      for k in range(self.K):
        interf_RZF[k,j,k,j] = interf_RZF[k,j,k,j] - signal_RZF[k,j]

    return signal_RZF, interf_RZF


  def computeSE_DL_poweralloc(self, rho, signal, interference):

    # Compute the prelog factor assuming only downlink transmission
    prelogFactor = (self.tau_c - self.tau_p) / (self.tau_c)

    # Prepare to save results
    SE = np.zeros((self.K,self.N))

    # Go through all cells
    for j in range(self.N):
      # Go through all UEs in cell j
      for k in range(self.K):
        # Compute the SE in Theorem 4.6 using the formulation in (7.1)
        SE[k,j] = prelogFactor*np.log2(1+(rho[k,j]*signal[k,j]) / (sum(sum(rho*interference[:,:,k,j])) + 1))
      
    return SE

  def MassiveMIMO_test(self,):

    nbrOfSetups = 30
    SE_RZF_equal = np.zeros((self.K,self.N, nbrOfSetups))
    Cumulative_SE_RZF_equal = np.zeros((self.K,self.N, nbrOfSetups))

    # Start massive MIMO channel setup
    R, channel_gain_db = self.channel_stat_setup(self.N, self.K, self.M, no_BS_per_dim=None, asd_degs=[10,], accuracy=2)
    R = R[:, :, :, :, :, 0]

    for n in range(nbrOfSetups):
      
      print("{} setups out of {}".format(n, nbrOfSetups))

      # Compute the normalized average channel gain, where the normalization is based on the noise power
      channelGainOverNoise = channel_gain_db - self.noiseVariancedBm

      # Compute the estimated channel matrix 'Hhat_MMSE' and generate the original channel matrix 'H'
      H, Hhat_MMSE = self.MMSE_channel_estimates(R, channelGainOverNoise)

      # RZF precoding (signal and interference part)
      signal, interf = self.compute_SINR_DL_rzf_precoding(H, Hhat_MMSE, self.nbrOfRealizations)
      print("===============================================================")
      print("signal RZF : {}".format(signal.sum()))
      print("interf RZF : {}".format(interf.sum()))

      # calcuation of SE each applyed PA algorithm (in this case equal power allocation for test)
      SE_RZF_equal = self.computeSE_DL_poweralloc(self.rhoEqual, signal, interf)
      Cumulative_SE_RZF_equal[:, :, n] = self.computeSE_DL_poweralloc(self.rhoEqual, signal, interf)
      print("SE_RZF_equal : {}".format(SE_RZF_equal.sum()))
      print("Cumulative_SE_RZF_equal : {}".format(Cumulative_SE_RZF_equal.sum()))
      print(" ")

    # (Here) calcuation of CDF
    SE_RZF_equal = SE_RZF_equal.flatten()

    plt.plot(np.sort(SE_RZF_equal),np.linspace(0,1,self.K * self.N),'-b',label='RZF')
    plt.legend(loc = 'upper left')
    plt.xlabel('SE per UE [bit/s/Hz]')
    plt.ylabel('CDF')
    plt.axis([0,7,0,1])
    plt.show()

    return SE_RZF_equal

scneario validation
--

In [ ]:
env = M_MIMOEnv(N = 4, M = 100, K = 5)
SE = env.MassiveMIMO_test()